<a href="https://colab.research.google.com/github/janaghoniem/Social-Media-Sentiment-Analysis/blob/main/ML_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import vstack
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import ast
import re
import pickle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Science/English preprocessing + pipeline & dataset/English_cleaned.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,normalized_tweet,tokens,trigrams_token,target
0,a that's a bummer. you shoulda got david carr ...,"[""'s"", 'bummer', '.', 'shoulda', 'got', 'david...","[('a', 'that', ""'s""), ('that', ""'s"", 'a'), (""'...",-1
1,is upset that he can't update his facebook by ...,"['upset', 'ca', ""n't"", 'updat', 'facebook', 't...","[('is', 'upset', 'that'), ('upset', 'that', 'h...",-1
2,i dived many times for the ball. managed to sa...,"['dive', 'mani', 'time', 'ball', '.', 'manag',...","[('i', 'dive', 'mani'), ('dive', 'mani', 'time...",-1
3,my whole body feels itchy and like its on fire,"['whole', 'bodi', 'feel', 'itchi', 'like', 'fi...","[('my', 'whole', 'bodi'), ('whole', 'bodi', 'f...",-1
4,"no, it's not behaving at all. i'm mad. why am ...","['no', ',', ""'s"", 'not', 'behav', '.', ""'m"", '...","[('no', ',', 'it'), (',', 'it', ""'s""), ('it', ...",-1


### 1) CatBoost ✅

In [3]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Keep the trigram list as is
X_tokens = df['trigrams_token'].tolist()  # List of lists of trigram strings
y = df['target']

# Vectorize using CountVectorizer with a custom analyzer that expects pre-tokenized input
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# CatBoost requires dense float32 arrays
X = X.toarray().astype(np.float32)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

# Split temp into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# -------------------- Baseline CatBoost --------------------
cat_model = CatBoostClassifier(verbose=0, random_state=42)
cat_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = cat_model.predict(X_train)
y_val_pred = cat_model.predict(X_val)
y_test_pred = cat_model.predict(X_test)

print(f"Baseline CatBoost Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline CatBoost Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline CatBoost Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'iterations': [100, 200]
}

grid_search = GridSearchCV(
    CatBoostClassifier(verbose=0, random_state=42),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit grid search only on training data
grid_search.fit(X_train, y_train)
best_cat = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_cat.predict(X_train)
y_val_pred_tuned = best_cat.predict(X_val)
y_test_pred_tuned = best_cat.predict(X_test)

print(f"\nTuned CatBoost Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned CatBoost Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned CatBoost Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


Baseline CatBoost Train Accuracy: 0.6515
Baseline CatBoost Validation Accuracy: 0.6309
Baseline CatBoost Test Accuracy: 0.6295

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.63      0.67      0.65    141567
           1       0.63      0.59      0.61    138451

    accuracy                           0.63    280018
   macro avg       0.63      0.63      0.63    280018
weighted avg       0.63      0.63      0.63    280018

Fitting 3 folds for each of 18 candidates, totalling 54 fits

Best hyperparameters: {'depth': 8, 'iterations': 200, 'learning_rate': 0.1}

Tuned CatBoost Train Accuracy: 0.6256
Tuned CatBoost Validation Accuracy: 0.6179
Tuned CatBoost Test Accuracy: 0.6164

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.61      0.67      0.64    141567
           1       0.62      0.56      0.59    138451

    accuracy              

### 2) Ridge Classifier

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# -------------------- Preprocessing --------------------
# Tokenized input
X_tokens = df['trigrams_token'].tolist()
y = df['target']

# Vectorize trigrams
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

# Split temp into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# -------------------- Baseline Ridge Classifier --------------------
ridge_model = RidgeClassifier()
ridge_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = ridge_model.predict(X_train)
y_val_pred = ridge_model.predict(X_val)
y_test_pred = ridge_model.predict(X_test)

print(f"Baseline Ridge Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline Ridge Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline Ridge Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]  # Regularization strength
}

grid_search = GridSearchCV(
    RidgeClassifier(),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)
best_ridge = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_ridge.predict(X_train)
y_val_pred_tuned = best_ridge.predict(X_val)
y_test_pred_tuned = best_ridge.predict(X_test)

print(f"\nTuned Ridge Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned Ridge Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned Ridge Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


Baseline Ridge Train Accuracy: 0.5983
Baseline Ridge Validation Accuracy: 0.5983
Baseline Ridge Test Accuracy: 0.5974

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.60      0.60      0.60    141567
           1       0.59      0.60      0.60    138451

    accuracy                           0.60    280018
   macro avg       0.60      0.60      0.60    280018
weighted avg       0.60      0.60      0.60    280018

Fitting 3 folds for each of 5 candidates, totalling 15 fits

Best hyperparameters: {'alpha': 0.01}

Tuned Ridge Train Accuracy: 0.5983
Tuned Ridge Validation Accuracy: 0.5983
Tuned Ridge Test Accuracy: 0.5974

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.60      0.60      0.60    141567
           1       0.59      0.60      0.60    138451

    accuracy                           0.60    280018
   macro avg       0.60      

### 3) Logistic regression ✅

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Keep the trigram list as is
X_tokens = df['trigrams_token'].tolist()  # List of lists of trigram strings
y = df['target']

# Vectorize using CountVectorizer with a custom analyzer that expects pre-tokenized input
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

# Split temp into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Train baseline Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000, random_state=42)
logreg_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = logreg_model.predict(X_train)
y_val_pred = logreg_model.predict(X_val)
y_test_pred = logreg_model.predict(X_test)

print(f"Baseline Logistic Regression Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline Logistic Regression Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline Logistic Regression Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],      # Inverse of regularization strength
    'penalty': ['l2'],                # L2 penalty (default and preferred)
    'solver': ['lbfgs'],              # Good solver for multiclass problems
}

grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000, random_state=42),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit grid search only on training data
grid_search.fit(X_train, y_train)
best_logreg = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_logreg.predict(X_train)
y_val_pred_tuned = best_logreg.predict(X_val)
y_test_pred_tuned = best_logreg.predict(X_test)

print(f"\nTuned Logistic Regression Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned Logistic Regression Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned Logistic Regression Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


Baseline Logistic Regression Train Accuracy: 0.5984
Baseline Logistic Regression Validation Accuracy: 0.5986
Baseline Logistic Regression Test Accuracy: 0.5975

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.60      0.60      0.60    141567
           1       0.59      0.60      0.59    138451

    accuracy                           0.60    280018
   macro avg       0.60      0.60      0.60    280018
weighted avg       0.60      0.60      0.60    280018

Fitting 3 folds for each of 5 candidates, totalling 15 fits

Best hyperparameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}

Tuned Logistic Regression Train Accuracy: 0.5984
Tuned Logistic Regression Validation Accuracy: 0.5986
Tuned Logistic Regression Test Accuracy: 0.5975

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.60      0.60      0.60    141567
           1       0.59  

### 4) Naive Bayes ✅

In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Keep the trigram list as is
X_tokens = df['trigrams_token'].tolist()  # List of lists of trigram strings
y = df['target']

# Vectorize using CountVectorizer with a custom analyzer that expects pre-tokenized input
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

# Split temp into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Train baseline Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = nb_model.predict(X_train)
y_val_pred = nb_model.predict(X_val)
y_test_pred = nb_model.predict(X_test)

print(f"Baseline NB Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline NB Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline NB Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]  # Laplace smoothing parameter
}

grid_search = GridSearchCV(
    MultinomialNB(),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit grid search only on training data
grid_search.fit(X_train, y_train)
best_nb = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_nb.predict(X_train)
y_val_pred_tuned = best_nb.predict(X_val)
y_test_pred_tuned = best_nb.predict(X_test)

print(f"\nTuned NB Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned NB Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned NB Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


Baseline NB Train Accuracy: 0.5817
Baseline NB Validation Accuracy: 0.5832
Baseline NB Test Accuracy: 0.5811

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.58      0.60      0.59    141567
           1       0.58      0.57      0.57    138451

    accuracy                           0.58    280018
   macro avg       0.58      0.58      0.58    280018
weighted avg       0.58      0.58      0.58    280018

Fitting 3 folds for each of 5 candidates, totalling 15 fits

Best hyperparameters: {'alpha': 10.0}

Tuned NB Train Accuracy: 0.5817
Tuned NB Validation Accuracy: 0.5832
Tuned NB Test Accuracy: 0.5811

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.58      0.60      0.59    141567
           1       0.58      0.57      0.57    138451

    accuracy                           0.58    280018
   macro avg       0.58      0.58      0.58    

### 5) XGBoost ✅

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier

# Assume df is already defined
X_tokens = df['trigrams_token'].tolist()  # List of lists of trigram strings
y = df['target']

# Vectorize using CountVectorizer with a custom analyzer
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train/Val/Test split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Baseline XGBoost Model
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)
xgb_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = xgb_model.predict(X_train)
y_val_pred = xgb_model.predict(X_val)
y_test_pred = xgb_model.predict(X_test)

print(f"Baseline XGBoost Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline XGBoost Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline XGBoost Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)
best_xgb = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_xgb.predict(X_train)
y_val_pred_tuned = best_xgb.predict(X_val)
y_test_pred_tuned = best_xgb.predict(X_test)

print(f"\nTuned XGBoost Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned XGBoost Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned XGBoost Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:20:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Baseline XGBoost Train Accuracy: 0.6415
Baseline XGBoost Validation Accuracy: 0.6224
Baseline XGBoost Test Accuracy: 0.6218

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.62      0.67      0.64    141567
           1       0.63      0.57      0.60    138451

    accuracy                           0.62    280018
   macro avg       0.62      0.62      0.62    280018
weighted avg       0.62      0.62      0.62    280018

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:55:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Best hyperparameters: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}

Tuned XGBoost Train Accuracy: 0.6705
Tuned XGBoost Validation Accuracy: 0.6282
Tuned XGBoost Test Accuracy: 0.6279

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.62      0.67      0.65    141567
           1       0.63      0.59      0.61    138451

    accuracy                           0.63    280018
   macro avg       0.63      0.63      0.63    280018
weighted avg       0.63      0.63      0.63    280018



### 6) LightGBM (Light Gradient Boosting Machine) ✅

In [5]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Keep the trigram list as is
X_tokens = df['trigrams_token'].tolist()  # List of lists of trigram strings
y = df['target']

# Vectorize using CountVectorizer with a custom analyzer that expects pre-tokenized input
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# Convert the data in the sparse matrix to float32
# LightGBM expects floating point data types.
X = X.astype(np.float32)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

# Split temp into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Train baseline LightGBM model
lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = lgbm_model.predict(X_train)
y_val_pred = lgbm_model.predict(X_val)
y_test_pred = lgbm_model.predict(X_test)

print(f"Baseline LightGBM Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline LightGBM Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline LightGBM Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'num_leaves': [31, 50, 100],       # number of leaves in one tree
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'max_depth': [-1, 10, 20]          # max tree depth (-1 means no limit)
}

grid_search = GridSearchCV(
    LGBMClassifier(random_state=42),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit grid search only on training data
grid_search.fit(X_train, y_train)
best_lgbm = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_lgbm.predict(X_train)
y_val_pred_tuned = best_lgbm.predict(X_val)
y_test_pred_tuned = best_lgbm.predict(X_test)

print(f"\nTuned LightGBM Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned LightGBM Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned LightGBM Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 415352, number of negative: 424700
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.291910 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1423
[LightGBM] [Info] Number of data points in the train set: 840052, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.494436 -> initscore=-0.022257
[LightGBM] [Info] Start training from score -0.022257


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Baseline LightGBM Train Accuracy: 0.6183
Baseline LightGBM Validation Accuracy: 0.6154
Baseline LightGBM Test Accuracy: 0.6127

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.61      0.66      0.63    141567
           1       0.62      0.56      0.59    138451

    accuracy                           0.61    280018
   macro avg       0.61      0.61      0.61    280018
weighted avg       0.61      0.61      0.61    280018

Fitting 3 folds for each of 81 candidates, totalling 243 fits


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 415352, number of negative: 424700
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.200675 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1423
[LightGBM] [Info] Number of data points in the train set: 840052, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.494436 -> initscore=-0.022257
[LightGBM] [Info] Start training from score -0.022257
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 10, '

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Tuned LightGBM Train Accuracy: 0.6826
Tuned LightGBM Validation Accuracy: 0.6325
Tuned LightGBM Test Accuracy: 0.6303

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.63      0.67      0.65    141567
           1       0.64      0.59      0.61    138451

    accuracy                           0.63    280018
   macro avg       0.63      0.63      0.63    280018
weighted avg       0.63      0.63      0.63    280018

